In [2]:
import torch 
import numpy as np

In [43]:
# create a fake sequence of 10 images, the shape that comes out from DALI
seq_len = 10
img_shape = (3, 2, 2)
img_seq = torch.zeros((seq_len,) + img_shape)
for i in range(seq_len):
    img = torch.ones(img_shape) * i
    img_seq[i] = img

In [54]:
img_seq.shape[1:]

torch.Size([3, 2, 2])

In [44]:
# our goal is to extract 5-frame sequences from this sequence
net_seq_len = 5
train_seq = torch.zeros((seq_len, net_seq_len, *img_shape), device=img_seq.device)
# define pads: start pad repeats the zeroth image twice. end pad repeats the last image twice.
# this is to give padding for the first and last frames of the sequence
pad_start = torch.tile(img_seq[0].unsqueeze(0), (2, 1,1,1))
pad_end = torch.tile(img_seq[-1].unsqueeze(0), (2, 1,1,1))
# pad the sequence
padded_seq = torch.cat((pad_start, img_seq, pad_end), dim=0)
#padded_seq = torch.cat((two_pad, img_seq, two_pad), dim=0)
for i in range(seq_len):
    # extract 5-frame sequences from the padded sequence
    train_seq[i] = padded_seq[i:i+net_seq_len]


In [53]:
# final image assertion
assert(torch.allclose(train_seq[-1][-1], img_seq[-1]))
assert(torch.allclose(train_seq[-1][-3], img_seq[-1]))
assert(~torch.allclose(train_seq[-1][-5], img_seq[-1]))

# image 0 assertion
assert(torch.allclose(train_seq[0][0], img_seq[0]))
assert(torch.allclose(train_seq[0][2], img_seq[0]))
assert(torch.allclose(train_seq[0][4], img_seq[2]))

# middle image assertion
assert(torch.allclose(train_seq[5][0], img_seq[3]))
assert(torch.allclose(train_seq[5][1], img_seq[4]))
assert(torch.allclose(train_seq[5][2], img_seq[5]))



In [17]:
img_seq[0].shape

torch.Size([3, 2, 2])

In [18]:
two_pad = torch.zeros(2, *img_shape)
two_pad.shape

torch.Size([2, 3, 2, 2])

In [19]:
padded_seq = torch.cat((two_pad, img_seq, two_pad), dim=0)
padded_seq.shape

torch.Size([14, 3, 2, 2])